In [7]:
import tensorflow as tf
g1=tf.Graph()
with g1.as_default():
    v=tf.get_variable("v",shape=[1],initializer=tf.zeros_initializer())
g2=tf.Graph()
with g2.as_default():
    v=tf.get_variable("v",shape=[1],initializer=tf.ones_initializer())

with tf.Session(graph=g1) as sess:
    tf.initialize_all_variables().run()
    with tf.variable_scope("",reuse=True):
        print(sess.run(tf.get_variable("v")))

with tf.Session(graph=g2) as sess:
    tf.initialize_all_variables().run()
    with tf.variable_scope("",reuse=True):
        print(sess.run(tf.get_variable("v")))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[ 0.]
Instructions for updating:
Use `tf.global_variables_initializer` instead.
[ 1.]


In [9]:
import tensorflow as tf
graph=tf.Graph()
with graph.as_default():
    a=tf.constant([1.0,2.0],name="a")
    b=tf.constant([2.0,3.0],name="b")
    result=tf.add(a,b,name="add")
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print(sess.run(result))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
[ 3.  5.]


In [14]:
import tensorflow as tf
graph=tf.Graph()
with graph.as_default():
    w1=tf.Variable(tf.random_normal([2,3],seed=1,stddev=1))
    w2=tf.Variable(tf.random_normal([3,1],seed=1,stddev=1))
    x=tf.constant([[0.7,0.9]])
    a=tf.matmul(x,w1)
    y=tf.matmul(a,w2)
    
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print(sess.run(y))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[ 3.95757794]]


In [16]:
import tensorflow as tf
graph=tf.Graph()
with graph.as_default():
    w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
    w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))
    x=tf.placeholder(tf.float32,shape=(1,2),name="input")
    
    a=tf.matmul(x,w1)
    y=tf.matmul(a,w2)
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print(sess.run(y,feed_dict={x:[[0.7,0.9]]}))
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[ 3.95757794]]


In [17]:
import tensorflow as tf
graph=tf.Graph()
with graph.as_default():
    w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
    w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))
    
    x=tf.placeholder(tf.float32,shape=(None,2),name="input")
    a=tf.matmul(x,w1)
    y=tf.matmul(a,w2)
    X_train=[[0.7,0.9],[0.1,0.4],[0.5,0.8]]
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print(sess.run(y,feed_dict={x:X_train}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[ 3.95757794]
 [ 1.15376544]
 [ 3.16749239]]


In [25]:
import tensorflow as tf
from numpy.random import RandomState
#定义数据batch的大小
batch_size=8
graph=tf.Graph()
with graph.as_default():
    #定义神经网络的参数
    w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
    w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))
    
    x=tf.placeholder(tf.float32,shape=(None,2),name="x_input")
    y_=tf.placeholder(tf.float32,shape=(None,1),name="y_input")
    
    #定义神经网络的前向传播过程
    a=tf.matmul(x,w1)
    y=tf.matmul(a,w2)
    
    #定义损失函数和反向传播的算法
    #损失函数用于刻画预测值和真实值之间的额差距
    cross_entropy=-tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))
    train_step=tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
    
    #通过随机数生成模拟数据集
    rdm=RandomState(1)
    dataset_szie=128
    X=rdm.rand(dataset_szie,2)
    Y=[[int(x1+x2<1)] for (x1,x2) in X]
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print(sess.run(w1))
    print(sess.run(w2))
    #设定训练的轮次
    STEPS=5000
    for i in range(STEPS):
        start=(i*batch_size)%dataset_szie
        end=min(start+batch_size,dataset_szie)
        
        #通过选取样本训练神经网络并更新参数
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        if(i%1000==0):
            #每隔一段时间计算所有数据上的交叉熵并输出
            total_cross_entropy=sess.run(cross_entropy,feed_dict={x:X,y_:Y})
            print("After %d training step(s) ,交叉熵为：%g" %(i,total_cross_entropy))
    print(sess.run(w1))
    print(sess.run(w2))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
[[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
After 0 training step(s) ,交叉熵为：0.0674925
After 1000 training step(s) ,交叉熵为：0.0163385
After 2000 training step(s) ,交叉熵为：0.00907547
After 3000 training step(s) ,交叉熵为：0.00714436
After 4000 training step(s) ,交叉熵为：0.00578471
[[-1.96182752  2.58235407  1.68203771]
 [-3.46817183  1.06982315  2.11788988]]
[[-1.82471502]
 [ 2.68546653]
 [ 1.41819501]]
